# KNN으로 KNOW 데이터 분석

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab Notebooks/2022 AI Study/데이콘/KNOW_직업추천

/content/drive/MyDrive/Colab Notebooks/2022 AI Study/데이콘/KNOW_직업추천


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from google.colab import drive
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import warnings
from numpy import dot
from numpy.linalg import norm
import re
import nltk
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm

nltk.download('punkt')
warnings.filterwarnings(action = 'ignore')


train2017_csv=pd.read_csv("./train/KNOW_2017.csv",low_memory=False)
train2018_csv=pd.read_csv("./train/KNOW_2018.csv",low_memory=False)
train2019_csv=pd.read_csv("./train/KNOW_2019.csv",low_memory=False)
train2020_csv=pd.read_csv("./train/KNOW_2020.csv",low_memory=False)

sample_submission_csv=pd.read_csv("./sample_submission.csv")
test2017_csv=pd.read_csv("./test/KNOW_2017_test.csv",low_memory=False)
test2018_csv=pd.read_csv("./test/KNOW_2018_test.csv",low_memory=False)
test2019_csv=pd.read_csv("./test/KNOW_2019_test.csv",low_memory=False)
test2020_csv=pd.read_csv("./test/KNOW_2020_test.csv",low_memory=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 2017년 데이터

In [ ]:
# 2017년 테스트 데이터의 인덱스는  0~ 9485
test2017_csv

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,4,4,5,4,5,5,6,4,...,1,52,1,,2,6,,,3400,
9482,9482,4,5,4,6,5,6,5,6,5,...,1,48,5,요업과,1,1,1,7000,,2400
9483,9483,3,2,1,,2,1,3,3,1,...,1,44,2,인문계,2,6,,,4500,
9484,9484,4,5,3,4,3,4,1,,1,...,1,44,4,컴퓨터공학,1,1,1,6000,,4000


In [ ]:
classifier2017 = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
label2017=train2017_csv['knowcode']
del train2017_csv['knowcode']
del train2017_csv['idx']
del train2017_csv['bq4_1a']
del train2017_csv['bq4_1b']
del train2017_csv['bq4_1c']
del train2017_csv['bq5_2']
del train2017_csv['bq19_1']
del train2017_csv['bq30']
del train2017_csv['bq31']
del train2017_csv['bq32']
del train2017_csv['bq33']
del train2017_csv['bq34'] 
del train2017_csv['bq38_1'] # bq4_1a, bq4_1b,bq4_1c,bq5_2,bq19_1,bq30,bq31,bq32,bq33,bq34,bq38_1


# test data
# original_test2017=test2017_csv # idx가 나중에 드롭되어서 원본 저장
del test2017_csv['idx']
del test2017_csv['bq4_1a']
del test2017_csv['bq4_1b']
del test2017_csv['bq4_1c']
del test2017_csv['bq5_2']
del test2017_csv['bq19_1']
del test2017_csv['bq30']
del test2017_csv['bq31']
del test2017_csv['bq32']
del test2017_csv['bq33']
del test2017_csv['bq34'] 
del test2017_csv['bq38_1'] 

In [ ]:
# 데이터 기본 전처리
train2017_csv.replace(' ',np.nan,inplace=True)
train2017_csv=train2017_csv.apply(pd.to_numeric)
train2017=train2017_csv.fillna(train2017_csv.mean())

test2017_csv.replace(' ',np.nan,inplace=True)
test2017_csv=test2017_csv.apply(pd.to_numeric)
test2017=test2017_csv.fillna(test2017_csv.mean())

In [ ]:
training_labels=label2017

In [ ]:
train2017_csv

,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,...,bq35,bq36,bq37,bq38,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,3,3.0,3,3.0,3,3.0,4,4.0,3,4.0,...,4,1,52,2,1,1,1.0,4000.0,NaN,2200.0
1,4,5.0,4,5.0,3,4.0,3,4.0,3,4.0,...,2,1,38,4,1,1,1.0,NaN,NaN,2400.0
2,3,4.0,3,4.0,3,4.0,5,6.0,4,5.0,...,2,1,50,4,1,1,1.0,4000.0,NaN,2400.0
3,3,3.0,3,3.0,3,5.0,4,5.0,4,6.0,...,4,1,42,4,1,1,1.0,7000.0,NaN,3500.0
4,4,5.0,3,4.0,3,4.0,4,5.0,3,4.0,...,2,1,51,4,1,1,1.0,4000.0,NaN,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,3,5.0,2,4.0,3,3.0,2,2.0,2,3.0,...,2,1,50,4,1,1,1.0,5200.0,NaN,1800.0
9482,5,5.0,5,5.0,5,5.0,3,4.0,4,5.0,...,2,1,37,4,1,1,1.0,4000.0,NaN,3000.0
9483,3,3.0,4,6.0,3,3.0,4,5.0,4,5.0,...,2,1,32,2,1,4,2.0,2700.0,NaN,1500.0
9484,3,5.0,3,5.0,4,5.0,3,4.0,3,5.0,...,2,2,40,4,1,1,1.0,6800.0,NaN,2500.0


In [ ]:
np_array=np.zeros((9486,9486))
training_points=train2017.to_numpy()
test_points=test2017.to_numpy()

In [ ]:
classifier2017.fit(training_points, training_labels)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
test_points.shape 

(9486, 143)

In [ ]:
ex=test_points[1,:].reshape(1,-1)

In [ ]:
def predict_2017KNN():
  # test 데이터의 행만큼 돌고
  for i in tqdm(range(test2017.shape[0])):
    # 구직자의 응답결과를 모델의 입력 형태에 맞추기
    test_row=test_points[i,:].reshape(1,-1)

    # 구직자의 응답결과 중 재직자와 가장 가까이 있는 이웃의 직업 반환
    recommand2017=classifier2017.predict(test_row)

    # # 추천받은 직업을 구직자의 직업란에 추천
    test2017_csv.at[i,'knowcode']=int(recommand2017)

In [ ]:
predict_2017KNN()

100%|██████████| 9486/9486 [00:32<00:00, 287.99it/s]


In [ ]:
test2017_csv

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,4.0,2,2.0,3,3.0,1,NaN,3,...,2,26,3,1,1,1.0,3000.0,NaN,2300.0,140401
1,1,5,5.0,3,5.0,5,5.0,5,5.0,4,...,1,57,4,1,1,1.0,5500.0,NaN,2500.0,158201
2,2,5,5.0,5,4.0,5,4.0,1,NaN,1,...,1,31,4,1,1,1.0,4300.0,NaN,4000.0,12201
3,3,4,5.0,5,6.0,4,6.0,3,4.0,4,...,1,35,6,1,1,1.0,4100.0,NaN,3000.0,133101
4,4,5,6.0,4,5.0,4,5.0,1,NaN,1,...,1,36,4,1,1,1.0,2800.0,NaN,2000.0,412003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,4.0,4,5.0,4,5.0,5,6.0,4,...,1,52,1,2,6,NaN,NaN,3400.0,NaN,561401
9482,9482,4,5.0,4,6.0,5,6.0,5,6.0,5,...,1,48,5,1,1,1.0,7000.0,NaN,2400.0,212101
9483,9483,3,2.0,1,NaN,2,1.0,3,3.0,1,...,1,44,2,2,6,NaN,NaN,4500.0,NaN,622304
9484,9484,4,5.0,3,4.0,3,4.0,1,NaN,1,...,1,44,4,1,1,1.0,6000.0,NaN,4000.0,13701


In [ ]:
test2017_csv['knowcode']=test2017_csv['knowcode'].astype('int')

In [ ]:
#2017년 테스트 데이터의 인덱스는  0~ 9485
#test2017_index=np.arange(9486)
# 원하는 위치에 칼럼 삽입 - (위치,칼럼명,값)
test2017_csv.insert(0,'idx',test2017_index)

In [ ]:
test2017_index

array([   0,    1,    2, ..., 9483, 9484, 9485])

In [ ]:
# 원하는 위치에 칼럼 삽입 - (위치,칼럼명,값)
test2017_csv.insert(0,'idx',test2017_index)

In [ ]:
col2017=['idx','knowcode']
target2017=test2017_csv[col2017]

## 2018년 데이터

In [ ]:
train2018_csv=pd.read_csv("./train/KNOW_2018.csv",low_memory=False)
test2018_csv=pd.read_csv("./test/KNOW_2018_test.csv",low_memory=False)

In [ ]:
test2018_index=test2018_csv['idx'] # 2018년 테스트 데이터의 인덱스는 9486~18568

In [ ]:
test2018_index.shape

(9069,)

In [ ]:
# 결측치 많은 칼럼 날리려 했는데 일단 패스
label2018=train2018_csv['knowcode']
test2018_index=test2018_csv['idx']

######## train data ###########
original2018=train2018_csv
del train2018_csv['idx']
del train2018_csv['knowcode']
del train2018_csv['bq4_1a']
del train2018_csv['bq4_1b']
del train2018_csv['bq4_1c']
del train2018_csv['bq5_2']
del train2018_csv['bq28_1']
del train2018_csv['bq29']
del train2018_csv['bq30']
del train2018_csv['bq31']
del train2018_csv['bq32']
del train2018_csv['bq33']
del train2018_csv['bq37_1']


######### test data #########
del test2018_csv['idx']
del test2018_csv['bq4_1a']
del test2018_csv['bq4_1b']
del test2018_csv['bq4_1c']
del test2018_csv['bq5_2']
del test2018_csv['bq28_1']
del test2018_csv['bq29']
del test2018_csv['bq30']
del test2018_csv['bq31']
del test2018_csv['bq32']
del test2018_csv['bq33']
del test2018_csv['bq37_1']


In [ ]:
unknown2018_train=train2018_csv.select_dtypes("object")
unknown2018_test=test2018_csv.select_dtypes("object")

# 속도를 위해 넘파이로 바꾸기
unknown2018_train=unknown2018_train.to_numpy()
unknown2018_test=unknown2018_test.to_numpy()

In [ ]:
# 학습 데이터 이상치 데이터 전처리
def train_unknown_processing():
  for i in tqdm(range(unknown2018_train.shape[0])): # 행별로 돌고
    for j in range(unknown2018_train.shape[1]): # 칼럼별로 돔
      try:
        unknown2018_train[i,j]=int(unknown2018_train[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2018_train[i,j]=3 # random 값 할당

# 테스트 데이터 이상치 데이터 전처리
def test_unknown_processing():
  for i in tqdm(range(unknown2018_test.shape[0])): # 행별로 돌고
    for j in range(unknown2018_test.shape[1]): # 칼럼별로 돔
      try:
        unknown2018_test[i,j]=int(unknown2018_test[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2018_test[i,j]=3 # random 값 할당

In [ ]:
train_unknown_processing()
test_unknown_processing()

100%|██████████| 9069/9069 [00:00<00:00, 27877.66it/s]


In [ ]:
# 전처리가 끝난 데이터를 넘파이 -> 데이터프레임 변환
processed_test2018=pd.DataFrame(unknown2018_test)
processed_train2018=pd.DataFrame(unknown2018_train)

train2018columns=['bq4', 'bq5', 'bq5_1', 'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq21',
       'bq221', 'bq222', 'bq223', 'bq231', 'bq234', 'bq235', 'bq241', 'bq242',
       'bq243', 'bq244', 'bq245', 'bq25', 'bq25_1', 'bq26_1', 'bq26_1a',
       'bq26_2', 'bq26_2a', 'bq26_3', 'bq26_3a', 'bq26_4', 'bq26_4a', 'bq28',
       'bq34', 'bq35', 'bq36', 'bq37', 'bq38', 'bq38_1', 'bq38_2', 'bq39',
       'bq40', 'bq41_1', 'bq41_2', 'bq41_3']

test2018columns=['bq4', 'bq5', 'bq5_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq221', 'bq222',
       'bq223', 'bq231', 'bq234', 'bq235', 'bq241', 'bq242', 'bq243', 'bq244',
       'bq245', 'bq25', 'bq25_1', 'bq26_1', 'bq26_1a', 'bq26_2a', 'bq26_3',
       'bq26_3a', 'bq26_4', 'bq26_4a', 'bq28', 'bq35', 'bq37', 'bq38',
       'bq38_1', 'bq38_2', 'bq39', 'bq41_1', 'bq41_2', 'bq41_3']

train2018_csv= train2018_csv.drop(train2018columns,axis=1)
test2018_csv= test2018_csv.drop(test2018columns,axis=1)

# 합치기 완료 !!
train2018=pd.concat([train2018_csv,processed_train2018],axis=1)
test2018=pd.concat([test2018_csv,processed_test2018],axis=1)

In [ ]:
# 데이터 기본 전처리 -- 전처리 성공 !!
train2018.replace(' ',np.nan,inplace=True)
train2018=train2018.apply(pd.to_numeric)
train2018=train2018.fillna(train2018.mean())

test2018.replace(' ',np.nan,inplace=True)
test2018=test2018.apply(pd.to_numeric)
test2018=test2018.fillna(test2018.mean())

In [ ]:
########## 빈 넘파이 행렬 만들고 기존의 train, test 데이터를 넘파이로 변환하기 #############
np_array=np.zeros((9072,9069)) #(train, test)

train2018_numpy=train2018.to_numpy()
test2018_numpy=test2018.to_numpy()

In [ ]:
classifier2018 = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
classifier2018.fit(train2018_numpy, label2018)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
def predict_2018KNN():
  # test 데이터의 행만큼 돌고
  for i in tqdm(range(test2018_csv.shape[0])):
    test_row=test2018_numpy[i,:].reshape(1,-1)
    recommand2018=classifier2018.predict(test_row)
    test2018_csv.at[i,'knowcode']=int(recommand2018)

In [ ]:
predict_2018KNN()

100%|██████████| 9069/9069 [00:29<00:00, 310.20it/s]


In [ ]:
test2018_csv

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,...,bq20,bq21,bq232,bq233,bq26_2,bq27,bq34,bq36,bq40,knowcode
0,9486,4,1,3,1,1,3,2,3,1,...,4.0,0.0,2.0,0.0,2.0,2.0,4.0,60.0,35.0,416204
1,9487,5,4,5,5,5,5,3,3,3,...,4.0,0.0,0.0,0.0,1.0,1.0,2.0,26.0,65.0,512301
2,9488,3,3,3,4,3,3,4,4,2,...,4.0,0.0,0.0,0.0,1.0,2.0,3.0,46.0,60.0,414102
3,9489,4,4,4,4,2,3,3,4,4,...,2.0,0.0,0.0,0.0,1.0,2.0,3.0,35.0,20.0,411101
4,9490,5,2,3,2,4,4,4,4,3,...,4.0,30.0,0.0,3.0,2.0,2.0,2.0,32.0,35.0,27101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9064,18564,5,4,2,2,2,4,4,3,3,...,3.0,0.0,0.0,0.0,1.0,1.0,4.0,38.0,72.0,33202
9065,18565,2,2,5,5,5,4,4,4,4,...,2.0,100.0,0.0,0.0,1.0,2.0,1.0,50.0,40.0,22201
9066,18566,4,4,3,2,2,3,4,4,4,...,3.0,100.0,0.0,0.0,1.0,2.0,3.0,42.0,40.0,140602
9067,18567,4,2,5,5,5,5,3,4,3,...,3.0,100.0,0.0,0.0,1.0,2.0,2.0,38.0,52.0,14201


In [ ]:
|len(test2018_index)

9082

In [ ]:
test2018_csv['knowcode']=test2018_csv['knowcode'].astype('int')

In [ ]:
# 원하는 위치에 칼럼 삽입 - (위치,칼럼명,값)
test2018_csv.insert(0,'idx',test2018_index)

In [ ]:
test2018_index

array([ 9486,  9487,  9488, ..., 18565, 18566, 18567])

In [ ]:
col2018=['idx','knowcode']
target2018=test2018_csv[col2018]

## 2019년 데이터

In [ ]:
test2019_csv

In [ ]:
# train data
test2019_index=test2019_csv['idx']

label2019=train2019_csv['knowcode']
del train2019_csv['idx']
del train2019_csv['knowcode']
del train2019_csv['bq4_1a']
del train2019_csv['bq4_1b']
del train2019_csv['bq4_1c']
del train2019_csv['bq5_2']
del train2019_csv['bq18_10']
del train2019_csv['bq20_1']
del train2019_csv['bq22']
del train2019_csv['bq23']
del train2019_csv['bq24']
del train2019_csv['bq27_1'] # bq4_1a,bq4_1b,bq4_1c,bq5_2,bq18_10,bq20_1,bq22,bq23,bq24,bq27_1



# test data
del test2019_csv['idx']
del test2019_csv['bq4_1a']
del test2019_csv['bq4_1b']
del test2019_csv['bq4_1c']
del test2019_csv['bq5_2']
del test2019_csv['bq18_10']
del test2019_csv['bq20_1']
del test2019_csv['bq22']
del test2019_csv['bq23']
del test2019_csv['bq24']
del test2019_csv['bq27_1']
#================================================================================================================

In [ ]:
label2019.shape

(8555,)

In [ ]:
# 객체인 칼럼만 떼와서 따로 저장(for. 전처리)
unknown2019_train=train2019_csv.select_dtypes("object")
unknown2019_test=test2019_csv.select_dtypes("object")

# 속도를 위해 따로 뗀 칼럼 넘파이로 바꾸기
unknown2019_train=unknown2019_train.to_numpy()
unknown2019_test=unknown2019_test.to_numpy()

In [ ]:
# 학습 데이터 이상치 데이터 전처리
def train2019_unknown_processing():
  for i in tqdm(range(unknown2019_train.shape[0])): # 행별로 돌고
    for j in range(unknown2019_train.shape[1]): # 칼럼별로 돔
      try:
        unknown2019_train[i,j]=int(unknown2019_train[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2019_train[i,j]=3 # random 값 할당

# 테스트 데이터 이상치 데이터 전처리
def test2019_unknown_processing():
  for i in tqdm(range(unknown2019_test.shape[0])): # 행별로 돌고
    for j in range(unknown2019_test.shape[1]): # 칼럼별로 돔
      try:
        unknown2019_test[i,j]=int(unknown2019_test[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2019_test[i,j]=3 # random 값 할당

In [ ]:
train2019_unknown_processing()
test2019_unknown_processing()

100%|██████████| 8554/8554 [00:00<00:00, 88798.77it/s]


In [ ]:

# 전처리가 끝난 데이터를 넘파이 -> 데이터프레임 변환
processed_test2019=pd.DataFrame(unknown2019_test)
processed_train2019=pd.DataFrame(unknown2019_train)

# 학습데이터 원본에서 전처리한 데이터로 교체(제거)
train2019columns=['bq5_1', 'bq6', 'bq7', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_4',
       'bq12_5', 'bq13_2', 'bq14_4', 'bq20', 'bq21_3', 'bq25', 'bq27',
       'bq28_1', 'bq28_2', 'bq29', 'bq30', 'bq31_1', 'bq31_2', 'bq31_3']

# 테스트 데이터에도 마찬가지로 작업
test2019columns=['bq5_1', 'bq19', 'bq20', 'bq21_3', 'bq26', 'bq27', 'bq28', 'bq28_1',
       'bq28_2', 'bq29', 'bq30', 'bq31_1', 'bq31_2', 'bq31_3']

train2019_csv= train2019_csv.drop(train2019columns,axis=1)
test2019_csv= test2019_csv.drop(test2019columns,axis=1)

# 합치기 완료 !!
train2019=pd.concat([train2019_csv,processed_train2019],axis=1)
test2019=pd.concat([test2019_csv,processed_test2019],axis=1)

In [ ]:
# 데이터 기본 전처리
train2019.replace(' ',np.nan,inplace=True)
train2019=train2019.apply(pd.to_numeric)
train2019=train2019.fillna(train2019.mean())

test2019.replace(' ',np.nan,inplace=True)
test2019=test2019.apply(pd.to_numeric)
test2019=test2019.fillna(test2019.mean())

In [ ]:
len(train2019.isnull().sum())

141

In [ ]:
########## 빈 넘파이 행렬 만들고 기존의 train, test 데이터를 넘파이로 변환하기 #############
np_array2019=np.zeros((8555,8554)) #(train, test)

train2019_numpy=train2019.to_numpy()
test2019_numpy=test2019.to_numpy()

In [ ]:
classifier2019 = KNeighborsClassifier(n_neighbors =3)

In [ ]:
classifier2019.fit(train2019_numpy, label2019)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
test2019_csv=pd.read_csv("./test/KNOW_2019_test.csv",low_memory=False)

In [ ]:
def predict_2019KNN():
  # test 데이터의 행만큼 돌고
  for i in tqdm(range(test2019_csv.shape[0])):
    test_row=test2019_numpy[i,:].reshape(1,-1)
    recommand2019=classifier2019.predict(test_row)
    test2019_csv.at[i,'knowcode']=int(recommand2019)

In [ ]:
predict_2019KNN()

100%|██████████| 8554/8554 [00:27<00:00, 305.65it/s]


In [ ]:
test2019_csv

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,5,5,5,4,4,5,4,5,5,...,경영,1,1,,1,45,6700,3800,,31102.0
1,18570,2,3,3,1,3,2,3,3,3,...,화학,1,1,,1,44,2700,2500,,703201.0
2,18571,5,4,5,4,4,4,4,5,4,...,건축공학,1,1,,1,40,6000,3200,,140206.0
3,18572,3,3,3,2,3,3,3,3,2,...,인문,1,1,,1,60,900,800,,532101.0
4,18573,5,4,5,3,3,3,3,4,5,...,기계공학과,1,1,,1,50,7000,5000,,151105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549,27122,3,4,5,2,4,4,4,5,5,...,화학공업과,1,3,,2,48,2400,1800,,155401.0
8550,27123,4,4,3,4,3,4,4,3,3,...,조소학과,1,3,,2,48,2800,1650,,307501.0
8551,27124,3,4,4,2,2,2,3,4,4,...,컴퓨터공학,1,1,,1,40,3500,2300,,612101.0
8552,27125,4,4,3,3,4,3,4,3,4,...,조소학과,1,1,,1,48,2800,1900,,231401.0


In [ ]:
test2019_csv['knowcode']=test2019_csv['knowcode'].astype('int')

In [ ]:
col2019=['idx','knowcode']
target2019=test2019_csv[col2019]

In [ ]:
target2019

,idx,knowcode
0,18569,31102
1,18570,703201
2,18571,140206
3,18572,532101
4,18573,151105
...,...,...
8549,27122,155401
8550,27123,307501
8551,27124,612101
8552,27125,231401


## 2020년 데이터

In [ ]:
test2020_csv

In [ ]:
######## train data ########### -bq4_1a,bq4_1b,bq4_1c,bq5_2,bq18_10, bq20_1 

# 학습데이터 원본 저장하는 이유 - row별 knowcode에 대한 정보를 알기위해
label2020=train2020_csv['knowcode']
test2020_index=test2020_csv['idx']

del train2020_csv['idx']
del train2020_csv['knowcode']
del train2020_csv['bq4_1a']
del train2020_csv['bq4_1b']
del train2020_csv['bq4_1c']
del train2020_csv['bq5_2']
del train2020_csv['bq18_10']
del train2020_csv['bq20_1']



######### test data #########
del test2020_csv['idx']
del test2020_csv['bq4_1a']
del test2020_csv['bq4_1b']
del test2020_csv['bq4_1c']
del test2020_csv['bq5_2']
del test2020_csv['bq18_10']
del test2020_csv['bq20_1']

#================================================================================================================

In [ ]:
unknown2020_train=train2020_csv.select_dtypes("object")
unknown2020_test=test2020_csv.select_dtypes("object")

# 속도를 위해 넘파이로 바꾸기
unknown2020_train=unknown2020_train.to_numpy()
unknown2020_test=unknown2020_test.to_numpy()

In [ ]:
# 학습 데이터 이상치 데이터 전처리
def train_unknown_processing():
  for i in tqdm(range(unknown2020_train.shape[0])): # 행별로 돌고
    for j in range(unknown2020_train.shape[1]): # 칼럼별로 돔
      try:
        unknown2020_train[i,j]=int(unknown2020_train[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2020_train[i,j]=3 # random 값 할당

        # 테스트 데이터 이상치 데이터 전처리
def test_unknown_processing():
  for i in tqdm(range(unknown2020_test.shape[0])): # 행별로 돌고
    for j in range(unknown2020_test.shape[1]): # 칼럼별로 돔
      try:
        unknown2020_test[i,j]=int(unknown2020_test[i,j])
      except ValueError:
        # 문자 이상치 데이터가 걸리는 곳 !!
        unknown2020_test[i,j]=3 # random 값 할당=

In [ ]:
train_unknown_processing()
test_unknown_processing()

100%|██████████| 8122/8122 [00:00<00:00, 24276.30it/s]


In [ ]:
# 전처리가 끝난 데이터를 넘파이 -> 데이터프레임 변환
processed_test2020=pd.DataFrame(unknown2020_test)
processed_train2020=pd.DataFrame(unknown2020_train)


# 학습데이터 원본에서 전처리한 데이터로 교체(제거)
# + bq5_1 은 객체로는 안 나오지만 거기서 걸리므로 같이 전처리
traincolumns=['saq1_2', 'saq2_2', 'saq3_2', 'saq4_2', 'saq5_2', 'saq6_2', 'saq7_2',
       'saq8_2', 'saq9_2', 'saq10_2', 'saq11_2', 'saq12_2', 'saq13_2',
       'saq14_2', 'saq15_2', 'saq16_2', 'saq17_2', 'saq18_2', 'saq19_2',
       'saq20_2', 'saq21_2', 'saq22_2', 'saq23_2', 'saq24_2', 'saq25_2',
       'saq26_2', 'saq27_2', 'saq28_2', 'saq29_2', 'saq30_2', 'saq31_2',
       'saq32_2', 'saq33_2', 'saq34_2', 'saq35_2', 'saq36_2', 'saq37_2',
       'saq38_2', 'saq39_2', 'saq40_2', 'saq41_2', 'saq42_2', 'saq43_2',
       'saq44_2', 'bq5_1', 'bq26_1', 'bq28', 'bq30_1', 'bq30_2', 'bq30_3']

train2020_csv= train2020_csv.drop(traincolumns,axis=1)

# 테스트 데이터에도 마찬가지로 작업
testcolumns=['saq1_2', 'saq2_2', 'saq3_2', 'saq4_2', 'saq5_2', 'saq6_2', 'saq7_2',
       'saq8_2', 'saq9_2', 'saq10_2', 'saq11_2', 'saq12_2', 'saq13_2',
       'saq14_2', 'saq15_2', 'saq16_2', 'saq17_2', 'saq18_2', 'saq19_2',
       'saq20_2', 'saq21_2', 'saq22_2', 'saq23_2', 'saq24_2', 'saq25_2',
       'saq26_2', 'saq27_2', 'saq28_2', 'saq29_2', 'saq30_2', 'saq31_2',
       'saq32_2', 'saq33_2', 'saq34_2', 'saq35_2', 'saq36_2', 'saq37_2',
       'saq38_2', 'saq39_2', 'saq40_2', 'saq41_2', 'saq42_2', 'saq43_2',
       'saq44_2', 'bq5_1', 'bq26_1', 'bq28', 'bq30_1', 'bq30_2', 'bq30_3']

test2020_csv= test2020_csv.drop(testcolumns,axis=1)

# 합치기 완료 !!
train2020=pd.concat([train2020_csv,processed_train2020],axis=1)
test2020=pd.concat([test2020_csv,processed_test2020],axis=1)

# 데이터 기본 전처리 -- 전처리 성공 !!
test2020.replace(' ',np.nan,inplace=True)
test2020=test2020.apply(pd.to_numeric)
test2020=test2020.fillna(test2020.mean())


In [ ]:
########## 빈 넘파이 행렬 만들고 기존의 train, test 데이터를 넘파이로 변환하기 #############
np_array2020=np.zeros((8122,8122)) #(train, test)

train2020_numpy=train2020.to_numpy()
test2020_numpy=test2020.to_numpy()

In [ ]:
classifier2020 = KNeighborsClassifier(n_neighbors = 3)
classifier2020.fit(train2020_numpy, label2020)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
test2020_csv=pd.read_csv("./test/KNOW_2020_test.csv",low_memory=False)

In [ ]:
def predict_2020KNN():
  # test 데이터의 행만큼 돌고
  for i in tqdm(range(test2020_csv.shape[0])):
    test_row=test2020_numpy[i,:].reshape(1,-1)
    recommand2020=classifier2020.predict(test_row)
    test2020_csv.at[i,'knowcode']=int(recommand2020)

In [ ]:
predict_2020KNN()

100%|██████████| 8122/8122 [00:29<00:00, 277.19it/s]


In [ ]:
test2020_csv

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,3,4,3,4,4,5,1,,1,...,2,문과,1,1,1,40,2300,2000,,511402
1,27128,4,5,4,5,4,5,3,4,4,...,4,한약학,2,5,1,40,,,5800,303002
2,27129,4,6,4,5,4,7,2,4,1,...,4,신문방송학,1,1,1,48,4700,3400,,32401
3,27130,3,4,3,4,3,3,1,,2,...,5,임학,1,1,1,40,4900,4500,,32201
4,27131,4,4,4,4,4,4,4,4,4,...,2,인문계,1,1,1,40,3200,2400,,25402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,35244,2,2,3,3,2,2,2,1,3,...,4,제어계측,1,1,1,40,4200,3200,,13701
8118,35245,5,6,5,6,4,6,3,3,3,...,4,일어일문,1,1,1,40,2600,2200,,231101
8119,35246,3,3,3,3,3,5,3,3,4,...,4,전기공학,1,1,1,45,4800,4100,,16101
8120,35247,3,5,3,4,4,5,4,4,3,...,4,화학,1,1,1,40,5000,4000,,15201


In [ ]:
test2020_csv['knowcode']=test2020_csv['knowcode'].astype('int')

In [ ]:
#test2020_csv['knowcode']=test2020_csv['knowcode'].astype('int')
col2020=['idx','knowcode']
target2020=test2020_csv[col2020]

In [ ]:
final2017=pd.DataFrame(target2017)
final2018=pd.DataFrame(target2018)
final2019=pd.DataFrame(target2019)
final2020=pd.DataFrame(target2020)

In [ ]:
KNN_submission = pd.concat([target2017,target2018,target2019,target2020], ignore_index=True)

In [ ]:
#KNN_submission['knowcode']=KNN_submission['knowcode'].astype('int')

In [ ]:
KNN_submission

,idx,knowcode
0,0,140401
1,1,158201
2,2,12201
3,3,133101
4,4,412003
...,...,...
35226,35244,13701
35227,35245,231101
35228,35246,16101
35229,35247,15201


In [ ]:
KNN_submission.to_csv("./adjust_k3_KNN.csv")

## 전체 train 데이터 concat해서 학습 시킨 후 predict

In [ ]:
train2017.columns=np.arange(143)
train2018.columns=np.arange(128)
train2019.columns=np.arange(141)
train2020.columns=np.arange(177)

test2017.columns=np.arange(143)
test2018.columns=np.arange(128)
test2019.columns=np.arange(141)
test2020.columns=np.arange(177)

In [ ]:
featureSum= pd.concat([train2017,train2018,train2019,train2020], ignore_index=True)

In [ ]:
labelSum= pd.concat([label2017,label2018,label2019,label2020], ignore_index=True)

In [ ]:
# 테스트 데이터 concat
testSum= pd.concat([test2017,test2018,test2019,test2020], ignore_index=True)

In [ ]:
print(test2017.shape)
print(test2018.shape)
print(test2019.shape)
print(test2020.shape)

(9486, 143)
(9069, 128)
(8554, 141)
(8122, 177)


In [ ]:
print(train2017.shape)
print(train2018.shape)
print(train2019.shape)
print(train2020.shape)

(9486, 143)
(9072, 128)
(8555, 141)
(8122, 177)


In [ ]:
featureSum.replace(' ',np.nan,inplace=True)
featureSum=featureSum.apply(pd.to_numeric)
featureSum=featureSum.fillna(featureSum.mean())

labelSum.replace(' ',np.nan,inplace=True)
labelSum=labelSum.apply(pd.to_numeric)
labelSum=labelSum.fillna(labelSum.mean())

testSum.replace(' ',np.nan,inplace=True)
testSum=testSum.apply(pd.to_numeric)
testSum=testSum.fillna(testSum.mean())

In [ ]:
featureSum=featureSum.to_numpy()
labelSum=labelSum.to_numpy()
testSum=testSum.to_numpy()

In [ ]:
KNNmodel = KNeighborsClassifier(n_neighbors = 5)

In [ ]:
KNNmodel.fit(featureSum, labelSum)

KNeighborsClassifier()

In [ ]:
KNNmodel.predict(ex)

In [ ]:
type(testSum)

pandas.core.frame.DataFrame

In [ ]:
# 이거 testSum 칼럼 shape이 이상하네
aaa=pd.DataFrame(testSum)
aaa

,0,1,2,3,4,5,6,7,8,9,...,167,168,169,170,171,172,173,174,175,176
0,3.0,4.0,2.0,2.0,3.0,3.0,1.0,3.834977,3.0,5.000000,...,3.635927,3.452105,3.825659,3.470943,2.978454,3.0,1.102807,3636.685176,2529.90101,689.060822
1,5.0,5.0,3.0,5.0,5.0,5.0,5.0,5.000000,4.0,5.000000,...,3.635927,3.452105,3.825659,3.470943,2.978454,3.0,1.102807,3636.685176,2529.90101,689.060822
2,5.0,5.0,5.0,4.0,5.0,4.0,1.0,3.834977,1.0,3.869464,...,3.635927,3.452105,3.825659,3.470943,2.978454,3.0,1.102807,3636.685176,2529.90101,689.060822
3,4.0,5.0,5.0,6.0,4.0,6.0,3.0,4.000000,4.0,5.000000,...,3.635927,3.452105,3.825659,3.470943,2.978454,3.0,1.102807,3636.685176,2529.90101,689.060822
4,5.0,6.0,4.0,5.0,4.0,5.0,1.0,3.834977,1.0,3.869464,...,3.635927,3.452105,3.825659,3.470943,2.978454,3.0,1.102807,3636.685176,2529.90101,689.060822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35226,2.0,3.0,2.0,2.0,3.0,3.0,1.0,2.000000,2.0,2.000000,...,5.000000,3.000000,4.000000,3.000000,3.000000,3.0,1.000000,4200.000000,3200.00000,3.000000
35227,5.0,5.0,4.0,3.0,3.0,4.0,3.0,4.000000,3.0,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,1.000000,2600.000000,2200.00000,3.000000
35228,3.0,3.0,3.0,3.0,4.0,4.0,1.0,3.000000,3.0,3.000000,...,3.000000,3.000000,4.000000,4.000000,3.000000,3.0,1.000000,4800.000000,4100.00000,3.000000
35229,3.0,3.0,4.0,4.0,3.0,3.0,3.0,4.000000,3.0,3.000000,...,3.000000,5.000000,4.000000,4.000000,1.000000,3.0,1.000000,5000.000000,4000.00000,3.000000


In [ ]:
from sklearn.impute import SimpleImputer
def predict_KNN():
  # test 데이터의 행만큼 돌고
  for i in tqdm(range(submission.shape[0])):
    test_row=testSum[i,:].reshape(1,-1)

    # 추출한 행벡터(test_row)를 모델에서 Knowcode를 예측
    recommand=KNNmodel.predict(test_row)
    submission.at[i,'knowcode']=int(recommand)

In [ ]:
predict_KNN()

100%|██████████| 35231/35231 [10:50<00:00, 54.18it/s]


In [ ]:
del submission['idx']

In [ ]:
submission.to_csv("./sumKNNpredict.csv")

In [ ]:
submission

,idx,knowcode
0,0,140401
1,1,131201
2,2,12201
3,3,12402
4,4,412003
...,...,...
35226,35226,13701
35227,35227,29501
35228,35228,16101
35229,35229,15201


In [ ]:
submission.insert(0,'idx',np.arange(35231))

In [ ]:
9486+9072+8555+8122

35235

In [ ]:
9486+9069+8554+8122

35231

In [ ]:
ls

코드/          corr_2020.csv       KNN_submission.csv     test/
corr_2018.csv  final_corr2017.csv  result.csv             train/
corr_2019.csv  final_submit.csv    sample_submission.csv


In [ ]:
submission=pd.read_csv("./final_submit.csv")

In [ ]:
del submission['Unnamed: 0']

In [ ]:
submission

,Unnamed: 0,idx,knowcode
0,0,0,811201
1,1,1,833001
2,2,2,31201
3,3,3,12402
4,4,4,306701
...,...,...,...
35226,35226,35244,612101
35227,35227,35245,231101
35228,35228,35246,153202
35229,35229,35247,15201


In [ ]:
# 제출파일의 형식의 값을 0으로 초기화
submission['knowcode']=0

In [ ]:
submission.to_csv('./sumKNNpredict.csv', index=False)